# Description

In this tutorial, we will learn how to load data MRI images saved in the `.nii` format using `nibabel` package. We will use the sample data that comes with the `bric_radiomics` package. The sample data is supposed to represent a lesion. After, we will compute the surface features using the `bric_radiomics` package. Next, we will visualize several statistical features of the surface measures. Finally, we will visualize the surface mesh of all the surface measures.

The tutorial will take the following steps:

1. Importing the packages
1. Configuring the logging to view output generated by the bric_radiomics package
1. Importing the `plotly` visualization package
1. Converting `.nii` files to a numpy array
1. Converting numpy array to a sanitized mask using bric_radiomics utility functions
1. Retrieve the surface measures as a pandas dataframe
1. Visualize the Matrix Scatterplot of all the surface measures.
1. Visualize the histograms of each surface measure.
1. Visualize the mesh of the lesion with the surface measure overlayed on each point on the mesh


In [ ]:
import bric_radiomics as br
import nibabel as ni
import plotly
import pandas as pd
import logging
FORMAT = '%(asctime)-15s %(levelname)s %(funcName)s  %(message)s'
logging.basicConfig(format=FORMAT, level=logging.DEBUG)

In [ ]:
from plotly.offline import download_plotlyjs, init_notebook_mode
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
init_notebook_mode()

In [ ]:
path = br.get_sample_nii_path()
ni_image = ni.load(path)
mri_image = ni_image.get_fdata()

In [ ]:
#set config values in this cell
vals = dict(
    voxel_spacing = (1.,1.,1.),
    gaussian_iterations = 3,
    gaussian_sigma = 2,
    marching_cubes_step_size = 1,
    clip_percent = .1
)

config = br.MorphologyConfig(**vals)

In [ ]:
mask_12 = br.convert_volume_into_mask(mri_image,merge_labels=[1,2])
mask_1 = br.convert_volume_into_mask(mri_image,merge_labels=[2])
mask_2 = br.convert_volume_into_mask(mri_image,merge_labels=[1])
lesion = br.compute_morphology_features(mask_12, config)
# lesion = br.compute_morphology_features(mask_1)
# lesion = br.compute_morphology_features(mask_2)

In [ ]:
df = lesion.to_DF()
df

In [ ]:
def matrix_scatter_fig(df):
    fig = px.scatter_matrix(df,
                            title = "Scatter Matrix for all surface features",
                            dimensions = ["shape_index", 
                                          "curvedness", 
                                          "sharpness", 
                                          "total_curvature"]
                           )
    fig.update_traces(diagonal_visible=False, 
                      showupperhalf=False)
    return fig

fig = matrix_scatter_fig(df)
fig.show()

In [ ]:
def histograms(df):
    fig = make_subplots(rows=2, cols=2)

    si_hg = go.Histogram(x=df["shape_index"],name="Shape Index")
    c_hg  = go.Histogram(x=df["curvedness"], name="Curvedness")
    s_hg  = go.Histogram(x=df["sharpness"], name="Sharpness")
    tc_hg = go.Histogram(x=df["total_curvature"], name="Total Curvature")

    fig.append_trace(si_hg,1,1)
    fig.append_trace(c_hg, 1,2)
    fig.append_trace(s_hg,2,1)
    fig.append_trace(tc_hg,2,2)
    fig.update_layout(
        title = {
            "text": "Surface Measure Histograms",
            'xanchor': 'center',
            'yanchor': 'top'
        })
    return fig
fig = histograms(df)
fig.show()

In [ ]:
def create_3dmesh(lesion, df, column_name):
    x_coordinates = df["x"]
    y_coordinates = df["y"]
    z_coordinates = df["z"]

    x_face_index = lesion.isosurface.faces[:, 0]
    y_face_index = lesion.isosurface.faces[:, 1]
    z_face_index = lesion.isosurface.faces[:, 2]
    
    vals = df[column_name]
        
    plotly_mesh = go.Mesh3d(
        x=x_coordinates,
        y=y_coordinates,
        z=z_coordinates,
        i=x_face_index,
        j=y_face_index,
        k=z_face_index,
        text=list(zip(x_coordinates,y_coordinates,z_coordinates,vals)),
        intensity=vals
    )
    return plotly_mesh

def all_meshes(lesion, df):
    plotly_3d_meshes = []
    columns = [
           "curvedness",
           "shape_index", 
           "sharpness",
           "total_curvature"
        ]
    for measure_name in columns:
        plotly_3d_mesh = create_3dmesh(lesion, df, measure_name)
        plotly_3d_meshes.append(plotly_3d_mesh)
    return plotly_3d_meshes


def generate_figure(plotly_3d_meshes):
    fig = go.Figure(data=plotly_3d_meshes)
    fig.update_layout(
        updatemenus=[
            dict(
                type="buttons",
                direction="right",
                active=0,
                x=0.57,
                y=1.2,
                buttons=list([
                    dict(label="curvedness",
                         method="update",
                         args=[{"visible": [True, False, False, False]},
                               {"title": "Curvedness"}]),
                    dict(label="shape index",
                         method="update",
                         args=[{"visible": [False, True, False, False]},
                               {"title": "Shape Index"}]),
                    dict(label="sharpness",
                         method="update",
                         args=[{"visible": [False, False, True, False]},
                               {"title": "Sharpness"}]),
                    dict(label="total curvature",
                         method="update",
                         args=[{"visible": [False, False, False, True]},
                               {"title": "Total Curvature"}]),
                ]),
            )
        ]
    )
    return fig

meshes = all_meshes(lesion, df)
fig = generate_figure(meshes)
fig.show()